In [7]:
import pandas as pd
import numpy as np
import os
folder = os.getcwd()

In [9]:
triplets = pd.read_table(folder+'\\data\\10000.txt', header=None, names=['user_id', 'song_id', 'listen_count'])
metadata = pd.read_csv(folder+'\\data\\metadata.csv')

In [ ]:
metadata

In [4]:
songs = pd.merge(triplets, metadata.drop_duplicates(['song_id']), on='song_id', how='left')

In [5]:
len(songs)

2000000

In [11]:
len(songs)

2000000

In [7]:
def full_name(title, artist):
    return (title + ' - ' + artist)
        
songs['name'] = songs.apply(lambda x: full_name(x['title'], x['artist_name']), axis=1)

In [8]:
songs['name']

0                                    The Cove - Jack Johnson
1                            Entre Dos Aguas - Paco De Lucia
2                                      Stronger - Kanye West
3                              Constellations - Jack Johnson
4                                Learn To Fly - Foo Fighters
                                 ...                        
1999995                 Ignorance (Album Version) - Paramore
1999996    Two Is Better Than One - Boys Like Girls featu...
1999997         What I've Done (Album Version) - Linkin Park
1999998                                   Up - Justin Bieber
1999999          Soil_ Soil (Album Version) - Tegan And Sara
Name: name, Length: 2000000, dtype: object

In [14]:
songs_grouped = songs.groupby(['name']).agg({'listen_count': 'count'}).reset_index()
total_listens = songs_grouped['listen_count'].sum()
songs_grouped['percentage']  = songs_grouped['listen_count'].div(total_listens)*100
songs_grouped = songs_grouped.sort_values(by='percentage', ascending=False)

In [15]:
songs_grouped

,name,listen_count,percentage
7127,Sehr kosmisch - Harmonia,8277,0.41385
9084,Undo - Björk,7032,0.35160
2068,Dog Days Are Over (Radio Edit) - Florence + Th...,6949,0.34745
9880,You're The One - Dwight Yoakam,6412,0.32060
6774,Revelry - Kings Of Leon,6145,0.30725
...,...,...,...
3526,Historia Del Portero - Ricardo Arjona,51,0.00255
7071,Scared - Three Days Grace,51,0.00255
2147,Don´t Leave Me Now - Amparanoia,50,0.00250
2991,Ghosts (Toxic Avenger Mix) - Ladytron,48,0.00240


In [30]:
songs_grouped.to_csv(folder+'\\songs_grouped.csv')

In [31]:
songs.to_csv(folder+'\\songs.csv')